# Concatenation of Pointclouds into one World Frame Tutorial


### Instead of visualizing only one point cloud at a time, we can simply aggregate all point cloud data frames into a single large one for visualization.

#### Load a sequence and its LiDAR point clouds

In [28]:
import pandaset

dataset = pandaset.DataSet('/data/PandaSet')
seq002 = dataset['002']
seq002.load_lidar()

Populating the interactive namespace from numpy and matplotlib


#### Use pandas concat method to concatenate all frames in selected slice

In [29]:
import pandas as pd

selected_data = seq002.lidar[::5]  # Take every 5th frame from sequence
_ = list(map(lambda xy: xy[1].insert(3,'f', xy[0]), enumerate(selected_data)))# Add column 'f' to each data frame in order

selected_data = pd.concat(selected_data)  # Concatenate in order

#### For better visualization we can scale the values in column `f` to `[0,1]` so it can be used for better point cloud colors.

In [30]:
import matplotlib.pyplot as plt

selected_data['f'] = (selected_data['f'] - selected_data['f'].min()) + 0.1*(selected_data['f'].max() - selected_data['f'].min())  # Add 10% of color range as base color (otherwise frame0 has white points)
selected_data['f'] /= selected_data['f'].max()

#### Now  we can use the concatenated point clouds with open3d visualizer

In [45]:
import open3d as o3d
import numpy as np

o3d_pc = o3d.geometry.PointCloud()
o3d_pc.points = o3d.utility.Vector3dVector(selected_data.to_numpy()[:, :3])
blue_colors = np.zeros((selected_data['f'].size,3))
blue_colors[:,2] = selected_data['f'].transpose()
o3d_pc.colors = o3d.utility.Vector3dVector(blue_colors)
o3d.visualization.draw_geometries([o3d_pc], window_name="concat frame")
